In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import math

In [3]:
#load datasets
am_df = pd.read_csv("data/american_stock.csv")
eu_df = pd.read_csv("data/european_index.csv")
rf_df = pd.read_csv("data/FEDFUNDS.csv")
stock_price_df = pd.read_csv("data/stock_prices.csv")

In [4]:
rf_df["DATE"] = pd.to_datetime(rf_df["DATE"], format="%Y-%m-%d")
rf_df.head()

,DATE,FEDFUNDS
0,2013-02-01,0.15
1,2013-03-01,0.14
2,2013-04-01,0.15
3,2013-05-01,0.11
4,2013-06-01,0.09


In [5]:
am_df = am_df[["date", "symbol", "exdate", "cp_flag", "strike_price", "best_bid", "best_offer", "exercise_style"]]
am_df["date"] = pd.to_datetime(am_df["date"], format="%Y%m%d")
am_df["exdate"] = pd.to_datetime(am_df["exdate"], format="%Y%m%d")
am_df["tte"] = am_df["exdate"] - am_df["date"]
am_df["rf_join_date"] = pd.to_datetime(am_df["date"].apply(lambda x: datetime.date(x.year, x.month, 1)))
am_df = pd.merge(am_df, rf_df, left_on="rf_join_date", right_on="DATE")
am_df.head()

,date,symbol,exdate,cp_flag,strike_price,best_bid,best_offer,exercise_style,tte,rf_join_date,DATE,FEDFUNDS
0,2015-01-02,AAPL 150102C100000,2015-01-02,C,100000,9.20,9.35,A,0 days,2015-01-01,2015-01-01,0.11
1,2015-01-02,AAPL 150102C101000,2015-01-02,C,101000,8.20,8.40,A,0 days,2015-01-01,2015-01-01,0.11
2,2015-01-02,AAPL 150102C102000,2015-01-02,C,102000,7.25,7.35,A,0 days,2015-01-01,2015-01-01,0.11
3,2015-01-02,AAPL 150102C103000,2015-01-02,C,103000,6.20,6.40,A,0 days,2015-01-01,2015-01-01,0.11
4,2015-01-02,AAPL 150102C104000,2015-01-02,C,104000,5.25,5.35,A,0 days,2015-01-01,2015-01-01,0.11


In [6]:
eu_df = eu_df[["date", "symbol", "exdate", "cp_flag", "strike_price", "best_bid", "best_offer", "exercise_style"]]
eu_df["date"] = pd.to_datetime(eu_df["date"], format="%Y%m%d")
eu_df["exdate"] = pd.to_datetime(eu_df["exdate"], format="%Y%m%d")
eu_df["tte"] = eu_df["exdate"] - eu_df["date"]
eu_df["rf_join_date"] = pd.to_datetime(eu_df["date"].apply(lambda x: datetime.date(x.year, x.month, 1)))
eu_df = pd.merge(eu_df, rf_df, left_on="rf_join_date", right_on="DATE")
eu_df.head()

,date,symbol,exdate,cp_flag,strike_price,best_bid,best_offer,exercise_style,tte,rf_join_date,DATE,FEDFUNDS
0,2015-01-02,NDX 150109C3100000,2015-01-09,C,3100000,1124.9,1134.0,E,7 days,2015-01-01,2015-01-01,0.11
1,2015-01-02,NDX 150109C3125000,2015-01-09,C,3125000,1099.9,1109.0,E,7 days,2015-01-01,2015-01-01,0.11
2,2015-01-02,NDX 150109C3150000,2015-01-09,C,3150000,1074.9,1084.0,E,7 days,2015-01-01,2015-01-01,0.11
3,2015-01-02,NDX 150109C3175000,2015-01-09,C,3175000,1049.9,1059.0,E,7 days,2015-01-01,2015-01-01,0.11
4,2015-01-02,NDX 150109C3200000,2015-01-09,C,3200000,1024.9,1034.0,E,7 days,2015-01-01,2015-01-01,0.11


In [112]:
#Current assumptions: risk-neutral and that FED rate is essentially the risk-free rate

def simple_european_binomial_model(n_steps, underlying_price, strike, tte, int_rate, call=True):
    """
    Returns the modeled price of a European options contract. 
    n_steps: number of steps
    underlying_price: initial stock price
    strike: strike price of option
    tte: time to expiration of contract (in days)
    int_rate: risk-free interest rate
    call: boolean True if call, False otherwise
    """
    dt = (tte/365)/n_steps
    u = 1.1 #I just used 10% expected vol for now
    d = 0.9 
    p = (math.exp(int_rate*dt)-d)/(u-d)
    
    stock_prices = [] #possible stock prices at the latest time
    options_prices = []
    for i in range(0,n_steps+1):
        stock_prices.append(underlying_price*(u**(n_steps-i))*(d**i))
        options_prices.append(max(0,stock_prices[i]-strike))
    
    #combine options data backwards in pairs
    counter = n_steps-1
    while counter > -1:
        for i in range(counter+1):
            price = math.exp(-int_rate*dt)*(p*options_prices[i] + (1-p)*options_prices[i+1])
            options_prices[i] = price
        counter-=1
        print(options_prices)
    return options_prices

In [114]:
# example values from the book
# one step inputs
simple_european_binomial_model(1,20,21,91,0.12)
# two step inputs
simple_european_binomial_model(2,20,21,182,0.12)

[0.6326361523710136, 0]
[2.0244356875872453, 0.0, 0]
[1.280731204117762, 0.0, 0]


[1.280731204117762, 0.0, 0]

In [106]:
def simple_american_binomial_model():
    pass